In [2]:
%matplotlib inline  
import matplotlib.pyplot as plt  


import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

 
# df=df_merged
# for column in df.columns:
#     for idx in df[column].index:
#         x = df.get_value(idx,column)
#         try:
#             x = unicode(x.encode('utf-8','ignore'),errors ='ignore') if type(x) == unicode else unicode(str(x),errors='ignore')
#             df.set_value(idx,column,x)
#         except Exception:
#             print 'encoding error: {0} {1}'.format(idx,column)
#             df.set_value(idx,column,'')
#             continue


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   

In [35]:
#####  i add university ranking

#  world_ranking	Institution	University	Location	national_ranking	Quality of Education	Alumni Employment	Quality of Faculty	Publications	Influence	Citations	Broad Impact	Patents	Score
#world_ranking	Institution	University	Location	national_ranking	Quality of Education	Alumni Employment	Quality of Faculty	Publications	Influence	Citations	Broad Impact	Patents	Score



path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/"
# input_file='list_top_100universities.tsv'
input_file = "world_ranking2_up_to_1000.csv"
df_univ_ranking=pd.read_csv(path+input_file, sep='\t',na_values=["NAN","-1","null"], usecols=["world_ranking","national_ranking","University"]) # set header=0 if i wanna pass it my own list of header names
df_univ_ranking.rename(columns={'University': 'Cleaned_university_name'}, inplace=True)

print df_univ_ranking.shape      #  1003





path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
df_complete=pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df_complete.drop(['Unnamed: 0', 'world_ranking','national_ranking'], axis=1, inplace=True)
print df_complete.shape      #  1403347, 



(1003, 3)
(1403347, 36)


In [14]:
df_complete.head()

,folder_id,num_folder_members,folder_creation_date,date_last_change,user_id,email_domain,num_adds,num_edits,num_deletes,major_content_type,...,Population_2010,Land-Sq-Mi,Density_Per_Sq_Mile,career_stage,median_num_publ,median_num_last_auth,folder_activity_GINI,category_total_publ,category_total_last_auth,category_total_cit
0,592732,4,2008-06-12,2012-12-25,77422,cornell.edu,NaN,NaN,NaN,office_docs,...,63886.0,125.959,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
1,592732,4,2008-06-12,2012-12-25,79630,cornell.edu,NaN,NaN,NaN,office_docs,...,63886.0,125.959,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
2,592732,4,2008-06-12,2012-12-25,77619,cornell.edu,80.0,72.0,0.0,office_docs,...,63886.0,125.959,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN
3,604794,2,2008-06-27,2012-12-23,86572,unc.edu,21798.0,40498.0,10660.0,pdf,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
4,680152,2,2008-08-26,2013-01-04,76579,cs.princeton.edu,644.0,0.0,112.0,pdf,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [36]:
df_merged = pd.merge(df_complete, df_univ_ranking, how='left',on='Cleaned_university_name') 
print "merged between master file and university ranking."
print df_merged.shape

merged between master file and university ranking.
(1403349, 38)


In [43]:
#df_merged[["Cleaned_university_name","world_ranking"]].head(10000)
df_selection_NAN=df_merged[pd.isnull(df_merged['world_ranking'])]
for item in df_selection_NAN["Cleaned_university_name"].unique():
    print item
len(df_merged["Cleaned_university_name"].unique())    

university of kansas medical center
eastern connecticut state university
california state university long beach
university of illinois
rutgers university
universita ca' foscari venezia
university of illinois chicago
universite de mons
santa clara university
western university
university of latvia
university of minnesota 
universidad de los andes
universidade federal do rio grande do sul
fordham university
universidad torcuato di tella
universitatsklinikum koln
university of wisconsin-eau claire
london school of economics and political science
wichita state university
embry-riddle aeronautical university
university of alaska
sheffield hallam university
technische universitat wien
university of boras
universitat wien
western washington university
california state university fullerton
vrije universiteit amsterdam
american university
universidad de puerto rico
elon university
university of mississippi
pacific university
iscte-university institute of lisbon
ebs universitat fur wirtschaft un

3401

In [44]:

new_merge_name="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/Dropbox_datafile_complete_new_ranking.csv"
df_merged.to_csv(new_merge_name, sep=';', encoding='utf-8')
print "written:", new_merge_name




name="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/Dropbox_datafile_complete_new_ranking.pickle"
with open(name,'wb') as f:
    pickle.dump(df_merged, f)

print "written:", name

written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/Dropbox_datafile_complete_new_ranking.csv
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/Dropbox_datafile_complete_new_ranking.pickle


In [45]:
df_merged.head(100)

,folder_id,num_folder_members,folder_creation_date,date_last_change,user_id,email_domain,num_adds,num_edits,num_deletes,major_content_type,...,Density_Per_Sq_Mile,career_stage,median_num_publ,median_num_last_auth,folder_activity_GINI,category_total_publ,category_total_last_auth,category_total_cit,world_ranking,national_ranking
0,592732,4,2008-06-12,2012-12-25,77422,cornell.edu,NaN,NaN,NaN,office_docs,...,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,10,8.0
1,592732,4,2008-06-12,2012-12-25,79630,cornell.edu,NaN,NaN,NaN,office_docs,...,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,10,8.0
2,592732,4,2008-06-12,2012-12-25,77619,cornell.edu,80.0,72.0,0.0,office_docs,...,507.196786,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,10,8.0
3,604794,2,2008-06-27,2012-12-23,86572,unc.edu,21798.0,40498.0,10660.0,pdf,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,40,25.0
4,680152,2,2008-08-26,2013-01-04,76579,cs.princeton.edu,644.0,0.0,112.0,pdf,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,9,7.0
5,693982,2,2008-09-08,2012-12-25,140646,columbia.edu,812.0,560.0,56.0,office_docs,...,68707.710010,jr,0.0,0.0,0.000000,1.0,1.0,2.0,6,4.0
6,756104,11,2008-09-14,2013-01-09,264014,cs.cmu.edu,NaN,NaN,NaN,photo,...,14473.955890,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,61,39.0
7,756104,11,2008-09-14,2013-01-09,205694,cs.cmu.edu,11480.0,0.0,0.0,photo,...,14473.955890,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,61,39.0
8,756104,11,2008-09-14,2013-01-09,260790,cs.pitt.edu,NaN,NaN,NaN,photo,...,0.000000,jr,0.0,0.0,0.666667,3.0,1.0,3.0,46,29.0
9,788418,2,2008-09-18,2013-01-09,188676,cornell.edu,419.0,84.0,28.0,photo,...,507.196786,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,10,8.0
